In [115]:
import pandas as pd 
from nltk import ngrams
import numpy as np

## Read the data from the files :

### Pre-Processed And Annotated YouTube Data :

In [229]:
data = pd.read_csv('../Data/PreProcessedYouTubeDataFileAndAnnotated.csv')

In [230]:
#data = data[["text","offensive/non offensive"]]
data = data.iloc[:,1:-1]
data.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,Algerian Dialect,auto_annotation
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p,NaN,n
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n,NaN,n
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n,NaN,n
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n,NaN,n
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n,NaN,n


### Dictionary with new bad words :
The new bad words have extracted from the offensive comments that have lenght >= 4.

In [231]:
dictionary = pd.read_csv('../Data/dictionary_with_new_bad_words.csv')

Convert the dictionary from DataFram type to Dict type

In [232]:
dictionary = dict(zip(dictionary["bad_word"], dictionary["frequency"]))

In [233]:
dictionary

{'مهبول': 1,
 'جايح': 8,
 'زبل': 3,
 'خرية': 3,
 'الله يلعنك': 2,
 'بلع فمك': 2,
 'متخلف': 1,
 'بغل': 8,
 'مسطاشة': 1,
 'حمارة': 3,
 'عطايا': 1,
 'حيوان': 3,
 'وي السلعة': 1,
 'نقتلك': 7,
 'تافه': 2,
 'قود': 8,
 'رخيس': 9,
 'حمار': 11,
 'تفو': 3,
 'عاقر': 1,
 'بلعه': 1,
 'مال ربك': 1,
 'سمينة': 1,
 'خرا': 3,
 'نيك امك': 2,
 'نيك مك': 3,
 'قهوي': 2,
 'يا الكعبة': 1,
 'كعبة': 2,
 'كافي': 4,
 'هايشة': 2,
 'ساقطة': 1,
 'معفون': 1,
 'يا الزح': 1,
 'متعرف والوا': 1,
 'معقد': 1,
 'كلب': 6,
 'بغلة': 2,
 'خماج': 5,
 'خامج': 3,
 'ينعل والديك': 2,
 'موسخ': 3,
 'رخيص': 1,
 'قحبة': 10,
 'حابس': 4,
 'نحيلك سروال': 1,
 'نيك اختك': 1,
 'زبي': 2,
 'اخرجلي طابي': 1,
 'شماتة': 2,
 'ڨليط': 2,
 'بلعيه': 1,
 'نت متفهمش': 1,
 'نحرقك': 3,
 'مقطعة': 2,
 'جايحة': 2,
 'قودي': 2,
 'ليك غير القمع': 1,
 'خامجة': 4,
 'انت دير هادي': 1,
 'طحان': 2,
 'فرخ': 4,
 'مشحاح': 1,
 'نقش': 5,
 'قحابجي': 1,
 'فايحة': 2,
 'عطاي': 6,
 'بومبا': 1,
 'بلع': 1,
 'يا ولد القحبة': 1,
 'بهلول': 4,
 'منغولي': 1,
 'ديوث': 2,
 'كلبة': 2,
 

### Annotate :
It is a function used to automaticly annotate a comment as offensive if it contains a bad word present in the dictionary 

In [234]:
def annotate(comment):
    dictionary = pd.read_csv('../Data/dictionary_with_new_bad_words.csv')
    dictionary = dict(zip(dictionary["bad_word"], dictionary["frequency"]))
    annotation = "n"
    for n in [1,2,3,4]:
        commentNgrams = ngrams(comment.split(), n)
        for grams in commentNgrams:
            key = ' '.join(grams)
            if key in dictionary : 
                annotation = "p"
    return annotation

In [235]:
data['auto_annotation'] = data["text"].apply(lambda x: annotate(x)) 


In [236]:
data['result'] = np.where(data["offensive/non offensive"] == data["auto_annotation"], 1, 0) 


In [237]:
data.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,Algerian Dialect,auto_annotation,result
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p,NaN,n,0
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n,NaN,n,1
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n,NaN,n,1
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n,NaN,n,1
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n,NaN,n,1


In [238]:
true_annotation = np.sum(data["result"])
print(true_annotation)

2541


In [239]:
len(data)

3306

In [240]:
true_annotation*100/ len(data)

76.86025408348458

# Using Annotation :
Using Annotation Function to get offensive Comments from the non annotated data

In [220]:
data_non_annotated = pd.read_csv('../Data/PreProcessedYouTubeDataFileAndNonAnnotated.csv')

In [222]:
data_non_annotated['auto_annotation'] = data_non_annotated["text"].apply(lambda x: annotate(x)) 
data_non_annotated = data_non_annotated.iloc[:,1:]
data_non_annotated.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,Algerian Dialect,auto_annotation
0,Mimi Mimj,http://www.youtube.com/channel/UCc4dkQ_XRKmgqd...,خوتي صلو على نبيكم السيده عندها حق وش تحكي بصح...,3,2020-10-21T14:22:18Z,NaN,NaN,n
1,Raziko Smail,http://www.youtube.com/channel/UCkqL7eRCKvTx2y...,راكم غاطين ستروا رواحكم شفوا,0,2020-10-22T21:14:06Z,NaN,NaN,n
2,nda nda,http://www.youtube.com/channel/UCQ9IzqkTPyKz1J...,والله يا مونيا لي راكي تهدري عليهم هوما لي طلع...,0,2020-10-21T14:21:38Z,NaN,NaN,n
3,SELSOULA,http://www.youtube.com/channel/UCfksSGorX_DMCr...,ماعندو مادخلك فينا حنا خاوة بنات دراري,0,2020-10-21T14:12:51Z,NaN,NaN,n
4,احمد بوقلقولة,http://www.youtube.com/channel/UCMkdl3UbXl0ZjM...,اسما يعني انتي ماشي بنادم,0,2020-10-21T14:07:27Z,NaN,NaN,n


In [223]:
cpt = 0
for index in range(len(data_non_annotated)) :
    comment = data_non_annotated.iloc[index]
    if comment["auto_annotation"] == "p":
        cpt += 1
        comment["offensive/non offensive"] = "p"
        data = data.append(comment)
        #print(comment["text"])
        #print("-------------------")


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [224]:
print(cpt)

728


In [225]:
data_non_annotated.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,Algerian Dialect,auto_annotation
0,Mimi Mimj,http://www.youtube.com/channel/UCc4dkQ_XRKmgqd...,خوتي صلو على نبيكم السيده عندها حق وش تحكي بصح...,3,2020-10-21T14:22:18Z,NaN,NaN,n
1,Raziko Smail,http://www.youtube.com/channel/UCkqL7eRCKvTx2y...,راكم غاطين ستروا رواحكم شفوا,0,2020-10-22T21:14:06Z,NaN,NaN,n
2,nda nda,http://www.youtube.com/channel/UCQ9IzqkTPyKz1J...,والله يا مونيا لي راكي تهدري عليهم هوما لي طلع...,0,2020-10-21T14:21:38Z,NaN,NaN,n
3,SELSOULA,http://www.youtube.com/channel/UCfksSGorX_DMCr...,ماعندو مادخلك فينا حنا خاوة بنات دراري,0,2020-10-21T14:12:51Z,NaN,NaN,n
4,احمد بوقلقولة,http://www.youtube.com/channel/UCMkdl3UbXl0ZjM...,اسما يعني انتي ماشي بنادم,0,2020-10-21T14:07:27Z,NaN,NaN,n


In [227]:
len(data)

3306

In [228]:
data.to_csv('../Data/PreProcessedYouTubeDataFileAndAnnotated.csv')
